In [1]:
import numpy as np
import json
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from utils.utils import seed_everything, create_dataset_from_json

In [5]:
obses, samples = create_dataset_from_json('/Disk3/yiming/episodes/1800/')

Loading data...


100%|██████████| 2080/2080 [01:38<00:00, 21.19it/s]


In [6]:
labels = [sample[-1] for sample in samples]
train, val = train_test_split(
        samples, test_size=0.1, random_state=0, stratify=labels)

In [7]:
print(len(train),len(val))

1994128 221570


In [ ]:
samples['train']=train
samples['val']=val

In [8]:
import pickle
with open("./data/obses.pkl",'wb') as f:
    pickle.dump(obses,f)
with open("./data/train_1800.pkl",'wb') as f:
    pickle.dump(train,f)
with open("./data/val_1800.pkl",'wb') as f:
    pickle.dump(val,f)

In [ ]:
# 保证不同map_size的sample个数大致相同
map_cnt = {
    'train':{},
    'val':{}
}
for k,v in samples.items():
    for obs_id, unit_id, action in v:
        map_size = obses[obs_id]['width']
        if not map_size in map_cnt[k].keys():
            map_cnt[k][map_size] = 0
        map_cnt[k][map_size]+=1
print(map_cnt)
train_cnt=min(map_cnt['train'].values())
val_cnt=min(map_cnt['val'].values())
print(train_cnt,val_cnt)

In [ ]:
from tqdm import tqdm
samples_dict={
    'train':{12:[],16:[],24:[],32:[]},
    'val':{12:[],16:[],24:[],32:[]}
}
for k,v in samples.items():
    for sample in tqdm(v):
        obs_id, unit_id, action=sample
        map_size = obses[obs_id]['width']
        samples_dict[k][map_size].append(sample)
for k,v in samples_dict.items():
    for key,value in v.items():
        print(k,key,len(value))

In [ ]:
import random
sample_cnt={'train':train_cnt,'val':val_cnt}
for k,v in samples_dict.items():
    for key,value in v.items():
        samples_dict[k][key]=random.sample(value,sample_cnt[k])

for k,v in samples_dict.items():
    for key,value in v.items():
        print(k,key,len(value))

In [ ]:
train_sample,val_sample=[],[]
for key,value in samples_dict['train'].items():
    train_sample+=value
for key,value in samples_dict['val'].items():
    val_sample+=value
print(len(train_sample))
print(len(val_sample))

In [ ]:
print(train_sample[0])


In [ ]:
import pickle
# with open("./data/obses.pkl",'wb') as f:
#     pickle.dump(obses,f)
with open("./data/train.pkl",'wb') as f:
    pickle.dump(train_sample,f)
with open("./data/val.pkl",'wb') as f:
    pickle.dump(val_sample,f)

In [ ]:
import pickle
with open("./data/train.pkl",'rb') as f:
    train_sample=pickle.load(f)
with open("./data/val.pkl",'rb') as f:
    val_sample=pickle.load(f)
with open("./data/obses.pkl",'rb') as f:
    obses=pickle.load(f)

In [ ]:
# 每个step的sample数量也要大致相同
train_sample_step={}
for sample in train_sample:
    obs_id, unit_id, action=sample
    obs=obses[obs_id]
    step=obs['step']
    if not step in train_sample_step.keys():
        train_sample_step[step]=[]
    train_sample_step[step].append(sample)
for k,v in sorted(train_sample_step.items(),key=lambda x: x[0]):
    print(k,len(v))

In [ ]:
import numpy as np
mean=int(np.mean([len(v) for v in train_sample_step.values()]))
mean

In [ ]:
import random
for k,v in train_sample_step.items():
    train_sample_step[k]=random.sample(v,min(len(v),mean))
for k,v in sorted(train_sample_step.items(),key=lambda x: x[0]):
    print(k,len(v))

In [ ]:
train_sample=[]
for k,v in train_sample_step.items():
    train_sample+=v

In [ ]:
print(len(train_sample))

In [ ]:
with open("./data/train.pkl",'wb') as f:
    pickle.dump(train_sample,f)

In [ ]:
samples=val_sample+train_sample
cnt=[0,0,0,0,0]
for obs_id, unit_id, label in tqdm(samples):
    cnt[label]+=1

In [ ]:
cnt=[i*-1 for i in cnt]
print(cnt)

In [ ]:
cnt=np.array(cnt)/np.abs(np.mean(cnt))
print(cnt)
cnt_exp=np.exp2(cnt)
sum=np.sum(cnt_exp)
print(sum)
cnt_exp=cnt_exp/sum*10
cnt_exp/min(cnt_exp)

In [ ]:
obs_cnt=[0]*360
for obs_id, unit_id, label in tqdm(samples):
    obs=obses[obs_id]
    step=obs['step']
    obs_cnt[step]+=1
np.sum(obs_cnt)

In [ ]:
from dataset.lux_dataset import make_input
import pickle
with open("./data/train.pkl",'rb') as f:
    train_sample=pickle.load(f)
with open("./data/val.pkl",'rb') as f:
    val_sample=pickle.load(f)
with open("./data/obses.pkl",'rb') as f:
    obses=pickle.load(f)

In [ ]:
for obs_id, unit_id, label in train_sample:
    obs=obses[obs_id]
    